In [1]:
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Dense, Dropout, Flatten, Activation, Concatenate, Lambda
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras import regularizers, activations
import os
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
train_datagen =  ImageDataGenerator(rescale=1.0/255, 
                                    rotation_range=10,
                                    zoom_range=0.1)

val_datagen =  ImageDataGenerator(rescale=1.0/255)

test_datagen =  ImageDataGenerator(rescale=1.0/255)

In [3]:
train_gen = train_datagen.flow_from_directory(directory='./train-copy/',
                                              target_size=(100, 100),
                                              color_mode='grayscale',
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=False,
                                              seed=0)
val_gen = val_datagen.flow_from_directory(directory='./val/',
                                          target_size=(100, 100),
                                          color_mode='grayscale',
                                          batch_size=32,
                                          class_mode='categorical',
                                          shuffle=False,
                                          seed=0)

test_gen = test_datagen.flow_from_directory(directory='./test/',
                                            target_size=(100, 100),
                                            color_mode='grayscale',
                                            batch_size=32,
                                            class_mode=None,
                                            shuffle=False,
                                            seed=0)

Found 8700 images belonging to 29 classes.
Found 28 images belonging to 29 classes.
Found 156 images belonging to 1 classes.


In [4]:
model = InceptionResNetV2(weights=None,
                          input_shape=(100,100,1),
                          classes=29)
model.compile(Adam(lr=0.0001), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
filepath = 'IR_weight.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', 
                        verbose=1, save_best_only=True, mode='max')

In [6]:
early = EarlyStopping(monitor='val_loss', 
                      mode='min', 
                      patience=4, restore_best_weights=True)
callbacks_list = [checkpoint, early]

In [7]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=29*300/32,          
                              validation_data=val_gen,
                              validation_steps=32, 
                              epochs=20, verbose=1,
                              callbacks=callbacks_list)

Epoch 1/20
223/271 [=======================>......] - ETA: 1:42 - loss: 3.7643 - accuracy: 0.0297

KeyboardInterrupt: 

In [ ]:
from keras.models import model_to_json
model_json = model.to_json()
with open("IR_model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
import json
with open('IR_history.json', 'w') as file:
    json.dump(history.history, file)

In [ ]:
# Training plots
epochs = [i for i in range(1, len(history.history['loss'])+1)]

plt.plot(epochs, history.history['loss'], color='blue', label="training_loss")
plt.plot(epochs, history.history['val_loss'], color='red', label="validation_loss")
plt.legend(loc='best')
plt.title('training')
plt.xlabel('epoch')
plt.savefig('./', bbox_inches='tight')
plt.show()

plt.plot(epochs, history.history['acc'], color='blue', label="training_accuracy")
plt.plot(epochs, history.history['val_acc'], color='red',label="validation_accuracy")
plt.legend(loc='best')
plt.title('validation')
plt.xlabel('epoch')
plt.savefig('./', bbox_inches='tight')
plt.show()